In [1]:
import pandas as pd
import numpy as np
import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision import models, transforms
# import matplotlib.pyplot as plt
# from sklearn.metrics import precision_recall_curve, auc
# import numpy as np
# import matplotlib.pyplot as plt
from tqdm.auto import tqdm
# from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer, SimilarityFunction
# from sklearn.metrics import roc_auc_score
import re
# import os
# import warnings
# warnings.filterwarnings("ignore")

In [2]:
tqdm.pandas()

# Загружаем данные 

In [3]:
%%time
path = "datasets/" # Ваш путь до директории с данными /path/to/data/
data = pd.read_csv(path + 'train_events.csv')
video = pd.read_csv(path + 'video_info_v2.csv')
targets = pd.read_csv(path + 'train_targets.csv')
all_events = pd.read_csv(path + 'all_events.csv')

# заполянем пропуски самыми частотными
freq_ua_client_type = all_events['ua_client_type'].value_counts().index[0]
freq_ua_client_name = all_events['ua_client_name'].value_counts().index[0]
all_events['ua_client_type'] = all_events['ua_client_type'].fillna(freq_ua_client_type)
all_events['ua_client_name'] = all_events['ua_client_name'].fillna(freq_ua_client_name)
all_events['ua_os'] = all_events['ua_os'].fillna('vpn')
data['ua_os'] = data['ua_os'].fillna('vpn')

# Преобразование даты и время в тип даты
all_events['event_timestamp'] = pd.to_datetime(all_events['event_timestamp'])
data['event_timestamp'] = pd.to_datetime(data['event_timestamp'])

# продолжительность видео duration - указан в миллисекнудах, а просмотр человеком (total_watchtime) в секндах приводим к одному виду
# video['duration'] = video['duration']/1000
all_events['total_watchtime'] = all_events['total_watchtime'] * 1000
data['total_watchtime'] = data['total_watchtime'] * 1000

data.shape, video.shape, targets.shape, all_events.shape

CPU times: total: 1min 25s
Wall time: 1min 45s


((1759616, 9), (481480, 5), (180012, 4), (8439624, 9))

In [5]:
%%time
from sklearn.model_selection import train_test_split

train_users_df, test_users_df = train_test_split(targets, stratify=(targets[['age_class', 'sex']]), test_size=0.1)
# train_users_df.shape, test_users_df.shape

data_video = data.merge(video, on='rutube_video_id')
train_users_df = data_video.merge(train_users_df, on='viewer_uid')
test_users_df = data_video.merge(test_users_df, on='viewer_uid')
del data_video

train_users_df.shape, test_users_df.shape

CPU times: total: 2.53 s
Wall time: 3.13 s


((1581685, 16), (177931, 16))

In [9]:
test_users_df.groupby(by='viewer_uid')[['category', 'title']]

In [24]:
%%time
category_test_users_df = test_users_df.groupby(by='viewer_uid')['category'].agg(lambda col: ' '.join(col))
title_test_users_df = test_users_df.groupby(by='viewer_uid')['title'].agg(lambda col: ' '.join(col))

category_train_users_df = train_users_df.groupby(by='viewer_uid')['category'].agg(lambda col: ' '.join(col))
title_train_users_df = train_users_df.groupby(by='viewer_uid')['title'].agg(lambda col: ' '.join(col))


category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

CPU times: total: 3.83 s
Wall time: 3.8 s


((18002,), (18002,), (162010,), (162010,))

In [31]:
%%time
category_test_users_df = category_test_users_df.reset_index().merge(targets, on='viewer_uid')
title_test_users_df = title_test_users_df.reset_index().merge(targets, on='viewer_uid')
category_train_users_df = category_train_users_df.reset_index().merge(targets, on='viewer_uid')
title_train_users_df = title_train_users_df.reset_index().merge(targets, on='viewer_uid')

category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

CPU times: total: 109 ms
Wall time: 108 ms


((18002, 5), (18002, 5), (162010, 5), (162010, 5))

In [32]:
# Функция для чистки текста
def clean_text(text):
    if pd.isna(text):
        return ''
    # оставляем только слова и цифры
    pattern = r"[a-zA-Z]+|[\d]+|[а-яА-Я]+"
    text = " ".join(re.findall(pattern, text.lower()))
    return text

In [36]:
%%time
category_test_users_df['clear_category'] = category_test_users_df['category'].apply(lambda x: clean_text(x))
category_train_users_df['clear_category'] = category_train_users_df['category'].apply(lambda x: clean_text(x))
title_test_users_df['clear_title'] = title_test_users_df['title'].apply(lambda x: clean_text(x))
title_train_users_df['clear_title'] = title_train_users_df['title'].apply(lambda x: clean_text(x))

category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

((18002, 6), (18002, 6), (162010, 6), (162010, 6))

In [40]:
category_test_users_df['sex_class'] = np.where(category_test_users_df['sex'] == 'male', 1, 0)
category_train_users_df['sex_class'] = np.where(category_train_users_df['sex'] == 'male', 1, 0)
title_test_users_df['sex_class'] = np.where(title_test_users_df['sex'] == 'male', 1, 0)
title_train_users_df['sex_class'] = np.where(title_train_users_df['sex'] == 'male', 1, 0)

category_test_users_df.shape, title_test_users_df.shape, category_train_users_df.shape, title_train_users_df.shape

((18002, 7), (18002, 7), (162010, 7), (162010, 7))

In [185]:
encode_model = SentenceTransformer('FacebookAI/xlm-roberta-large')
encode_model

No sentence-transformers model found with name FacebookAI/xlm-roberta-large. Creating a new one with mean pooling.


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

# Logistic Regression

In [49]:
%%time
title_test_users_df['embeddings'] = title_test_users_df['clear_title'].progress_apply(encode_model.encode)
title_train_users_df['embeddings'] = title_train_users_df['clear_title'].progress_apply(encode_model.encode)

title_test_users_df.shape, title_train_users_df.shape

  0%|          | 0/18002 [00:00<?, ?it/s]

  0%|          | 0/162010 [00:00<?, ?it/s]

CPU times: total: 1h 21min 26s
Wall time: 1h 25min 26s


((18002, 8), (162010, 8))

In [50]:
# %%time
# title_test_users_df.to_parquet('title_test_users_df.parquet')
# title_train_users_df.to_parquet('title_train_users_df.parquet')
# title_test_users_df.shape, title_train_users_df.shape

CPU times: total: 9.06 s
Wall time: 11.7 s


((18002, 8), (162010, 8))

In [64]:
union_title_users_df = pd.concat([title_test_users_df, title_train_users_df])
union_title_users_df.shape

(180012, 8)

In [186]:
%%time
category_test_users_df['embeddings'] = category_test_users_df['clear_category'].progress_apply(encode_model.encode)
category_train_users_df['embeddings'] = category_train_users_df['clear_category'].progress_apply(encode_model.encode)

category_test_users_df.shape, category_train_users_df.shape

  0%|          | 0/18002 [00:00<?, ?it/s]

  0%|          | 0/162010 [00:00<?, ?it/s]

CPU times: total: 56min 42s
Wall time: 57min 45s


((18002, 8), (162010, 8))

In [187]:
# %%time
# category_test_users_df.to_parquet('category_test_users_df.parquet')
# category_train_users_df.to_parquet('category_train_users_df.parquet')
# category_test_users_df.shape, category_train_users_df.shape

CPU times: total: 6.42 s
Wall time: 6.48 s


((18002, 8), (162010, 8))

In [188]:
union_category_users_df = pd.concat([category_test_users_df, category_train_users_df])
union_category_users_df.shape

(180012, 8)

# Генерируем фичи на базе title

### Берем отложенную выборку для локлаьного тесте

In [65]:
train_users_df = pd.read_csv('train_viewer_uid_v2.csv')
test_users_df = pd.read_csv('test_viewer_uid_v2.csv')

train_users_df.shape, test_users_df.shape

((1505791, 2), (170274, 2))

In [128]:
TRAIN_title_users_df = union_title_users_df[union_title_users_df['viewer_uid'].isin(train_users_df['viewer_uid'])]
TEST_title_users_df = union_title_users_df[union_title_users_df['viewer_uid'].isin(test_users_df['viewer_uid'])]

TRAIN_title_users_df.shape, TEST_title_users_df.shape

((155197, 8), (17221, 8))

# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео

In [133]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def calc_cv_target(TRAIN_title_users_df:pd.DataFrame, TEST_title_users_df:pd.DataFrame, target_col:str = 'sex_class'):
    
    X = TRAIN_title_users_df['embeddings'].values
    y = TRAIN_title_users_df[target_col].values
    TEST_X = TEST_title_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель логистической регрессии
    model = LogisticRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем вероятности для валидационных данных (берем вероятность класса 1)
        predicted_proba = model.predict_proba(list(X_valid))[:, 1]
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted_proba
        TEST_title_users_df[f'logistic_oof_{target_col}_{i}'] = model.predict_proba(list(TEST_X))[:, 1]
        print("Logistic Regression Accuracy:",metrics.accuracy_score(y_valid, predicted))
        print("Logistic Regression Precision:",metrics.precision_score(y_valid, predicted))
        print("Logistic Regression Recall:",metrics.recall_score(y_valid, predicted))
        print("Logistic Regression f1_score:",metrics.f1_score(y_valid, predicted))    

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_title_users_df[f'logistic_oof_{target_col}'] = oof_predictions

    temp_cols = [f'logistic_oof_{target_col}_{i}' for i in range(n_folds)]
    TEST_title_users_df[f'logistic_oof_{target_col}'] = TEST_title_users_df[temp_cols].mean(axis=1)
    TEST_title_users_df = TEST_title_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_title_users_df, TEST_title_users_df 

TRAIN_title_users_df, TEST_title_users_df = calc_cv_target(TRAIN_title_users_df, TEST_title_users_df, target_col='sex_class')
TRAIN_title_users_df.shape, TEST_title_users_df.shape

0it [00:00, ?it/s]

C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<timed exec>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Logistic Regression Accuracy: 0.7368556701030928
Logistic Regression Precision: 0.7246999935827504
Logistic Regression Recall: 0.7443807263858677
Logistic Regression f1_score: 0.7344085322234506


C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<timed exec>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Logistic Regression Accuracy: 0.7376288659793815
Logistic Regression Precision: 0.7249791893449447
Logistic Regression Recall: 0.7462922681431678
Logistic Regression f1_score: 0.7354813563726127


...

Logistic Regression Accuracy: 0.7446760527078836
Logistic Regression Precision: 0.7320056352459017
Logistic Regression Recall: 0.753477028541296
Logistic Regression f1_score: 0.7425861564946243


C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.7409710364380295
Logistic Regression Precision: 0.7264642358023122
Logistic Regression Recall: 0.75385629531971
Logistic Regression f1_score: 0.7399068322981367
CPU times: total: 6min 55s
Wall time: 46.5 s


<timed exec>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


((155197, 9), (17221, 9))

In [179]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

def calc_cv_target_regr(TRAIN_title_users_df:pd.DataFrame, TEST_title_users_df:pd.DataFrame, target_col:str = 'age'):
    
    X = TRAIN_title_users_df['embeddings'].values
    y = TRAIN_title_users_df[target_col].values
    TEST_X = TEST_title_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель линейной  регрессии
    model = LinearRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем возраст
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted
        TEST_title_users_df[f'linear_oof_{target_col}_{i}'] = model.predict(list(TEST_X))
        print("Logistic Regression MAE:",metrics.mean_absolute_error(y_valid, predicted))

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_title_users_df[f'linear_oof_{target_col}'] = oof_predictions

    temp_cols = [f'linear_oof_{target_col}_{i}' for i in range(n_folds)]
    TEST_title_users_df[f'linear_oof_{target_col}'] = TEST_title_users_df[temp_cols].mean(axis=1)
    TEST_title_users_df = TEST_title_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_title_users_df, TEST_title_users_df 

TRAIN_title_users_df, TEST_title_users_df = calc_cv_target_regr(TRAIN_title_users_df, TEST_title_users_df, target_col='age')
TRAIN_title_users_df.shape, TEST_title_users_df.shape

0it [00:00, ?it/s]

Logistic Regression MAE: 6.630822678140758
Logistic Regression MAE: 6.659621556456556
Logistic Regression MAE: 6.659132083915067
Logistic Regression MAE: 6.657421340101746
Logistic Regression MAE: 6.665511999975495
CPU times: total: 54.4 s
Wall time: 19.1 s


<timed exec>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


((155197, 10), (17221, 10))

In [182]:
%%time
# Сохраняем фичи полученные через кросс валидацию
TRAIN_title_users_df[['viewer_uid', 'logistic_oof_sex_class', 'linear_oof_age']].to_csv(f'cv_TRAIN_title_users_df.csv')
TEST_title_users_df[['viewer_uid', 'logistic_oof_sex_class', 'linear_oof_age']].to_csv(f'cv_TEST_title_users_df.csv')

CPU times: total: 344 ms
Wall time: 343 ms


In [189]:
TRAIN_title_users_df.shape, TEST_title_users_df.shape

((155197, 10), (17221, 10))

# Генерируем фичи на базе category

In [192]:
TRAIN_category_users_df = union_category_users_df[union_category_users_df['viewer_uid'].isin(train_users_df['viewer_uid'])]
TEST_category_users_df = union_category_users_df[union_category_users_df['viewer_uid'].isin(test_users_df['viewer_uid'])]

TRAIN_category_users_df.shape, TEST_category_users_df.shape

((155197, 8), (17221, 8))

In [193]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def calc_cv_target(TRAIN_category_users_df:pd.DataFrame, TEST_category_users_df:pd.DataFrame, target_col:str = 'sex_class'):
    
    X = TRAIN_category_users_df['embeddings'].values
    y = TRAIN_category_users_df[target_col].values
    TEST_X = TEST_category_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель логистической регрессии
    model = LogisticRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем вероятности для валидационных данных (берем вероятность класса 1)
        predicted_proba = model.predict_proba(list(X_valid))[:, 1]
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted_proba
        TEST_category_users_df[f'logistic_oof_cat_{target_col}_{i}'] = model.predict_proba(list(TEST_X))[:, 1]
        print("Logistic Regression Accuracy:",metrics.accuracy_score(y_valid, predicted))
        print("Logistic Regression Precision:",metrics.precision_score(y_valid, predicted))
        print("Logistic Regression Recall:",metrics.recall_score(y_valid, predicted))
        print("Logistic Regression f1_score:",metrics.f1_score(y_valid, predicted))    

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_category_users_df[f'logistic_oof_cat_{target_col}'] = oof_predictions

    temp_cols = [f'logistic_oof_cat_{target_col}_{i}' for i in range(n_folds)]
    TEST_category_users_df[f'logistic_oof_cat_{target_col}'] = TEST_category_users_df[temp_cols].mean(axis=1)
    TEST_category_users_df = TEST_category_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_category_users_df, TEST_category_users_df 

TRAIN_category_users_df, TEST_category_users_df = calc_cv_target(TRAIN_category_users_df, TEST_category_users_df, target_col='sex_class')
TRAIN_category_users_df.shape, TEST_category_users_df.shape

0it [00:00, ?it/s]

C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<timed exec>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Logistic Regression Accuracy: 0.7104703608247422
Logistic Regression Precision: 0.6964671495742788
Logistic Regression Recall: 0.722496869026432
Logistic Regression f1_score: 0.7092432624801838


C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<timed exec>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Logistic Regression Accuracy: 0.712048969072165
Logistic Regression Precision: 0.6921511807139774
Logistic Regression Recall: 0.7399644057741744
Logistic Regression f1_score: 0.715259636827015


...

Logistic Regression Accuracy: 0.7133606108444215
Logistic Regression Precision: 0.6954273610765014
Logistic Regression Recall: 0.7358117460945225
Logistic Regression f1_score: 0.7150498030298178


C:\Users\Profit77\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.7123940848609813
Logistic Regression Precision: 0.7002373468471358
Logistic Regression Recall: 0.7195781147000659
Logistic Regression f1_score: 0.709776000520173
CPU times: total: 6min 36s
Wall time: 44.5 s


<timed exec>:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


((155197, 9), (17221, 9))

In [194]:
%%time
# Генерируем фичи через кроссвалидацию на базе названий просмотрренных видео
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression

def calc_cv_target_regr(TRAIN_category_users_df:pd.DataFrame, TEST_category_users_df:pd.DataFrame, target_col:str = 'age'):
    
    X = TRAIN_category_users_df['embeddings'].values
    y = TRAIN_category_users_df[target_col].values
    TEST_X = TEST_category_users_df['embeddings'].values

    # Количество фолдов для кросс-валидации
    n_folds = 5

    # Создаем объект StratifiedKFold для кросс-валидации
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=53)

    # Создаем массив для хранения OOF предсказаний
    oof_predictions = np.zeros(len(X))

    # Модель линейной  регрессии
    model = LinearRegression()

    # Проходим по каждому фолду
    for i, (train_idx, valid_idx) in enumerate(tqdm(skf.split(X, y))):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        # Обучаем модель на данных фолда
        model.fit(list(X_train), list(y_train))

        # Предсказываем возраст
        predicted = model.predict(list(X_valid))
        oof_predictions[valid_idx] = predicted
        TEST_category_users_df[f'linear_oof_cat_{target_col}_{i}'] = model.predict(list(TEST_X))
        print("Logistic Regression MAE:",metrics.mean_absolute_error(y_valid, predicted))

    # Теперь oof_predictions содержит предсказания для всех данных, которые можно использовать как новую фичу
    TRAIN_category_users_df[f'linear_oof_cat_{target_col}'] = oof_predictions

    temp_cols = [f'linear_oof_cat_{target_col}_{i}' for i in range(n_folds)]
    TEST_category_users_df[f'linear_oof_cat_{target_col}'] = TEST_category_users_df[temp_cols].mean(axis=1)
    TEST_category_users_df = TEST_category_users_df.drop(columns=temp_cols, errors='ignore')

    return TRAIN_category_users_df, TEST_category_users_df 

TRAIN_category_users_df, TEST_category_users_df = calc_cv_target_regr(TRAIN_category_users_df, TEST_category_users_df, target_col='age')
TRAIN_category_users_df.shape, TEST_category_users_df.shape

0it [00:00, ?it/s]

Logistic Regression MAE: 6.941442029623641
Logistic Regression MAE: 6.923612660968427
Logistic Regression MAE: 6.942991416238241
Logistic Regression MAE: 6.944619879702163
Logistic Regression MAE: 6.938686427445121
CPU times: total: 52.2 s
Wall time: 18.6 s


<timed exec>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


((155197, 10), (17221, 10))

In [196]:
%%time
# Сохраняем фичи полученные через кросс валидацию
TRAIN_category_users_df[['viewer_uid', 'logistic_oof_cat_sex_class', 'linear_oof_cat_age']].to_csv(f'cv_TRAIN_category_users_df.csv')
TEST_category_users_df[['viewer_uid', 'logistic_oof_cat_sex_class', 'linear_oof_cat_age']].to_csv(f'cv_TEST_category_users_df.csv')

TRAIN_category_users_df.shape, TEST_category_users_df.shape 

CPU times: total: 328 ms
Wall time: 337 ms


((155197, 10), (17221, 10))